In [1]:
import pandas as pd
import numpy as np
import math
import collections
import re
import sys
import itertools

In [54]:
## swap position x with position y - letters at x and y should be swapped
## 
def swapchar(positions,strin):
    posx = sorted(positions)[0]
    posy = sorted(positions)[1]
    strout = strin[:posx] + strin[posy] + strin[posx+1:posy] + strin[posx] + strin[posy+1:]
    return strout
    
## swap letter x with letter y
def swapletter(letterx,lettery,strin):
    posx = re.search(letterx,strin).start()
    posy = re.search(lettery,strin).start()
    return swapchar((posx,posy),strin)
    
def rotatedir(direction,steps,strin):
    if direction == 'right':
        steps *= -1
    return strin[steps::] + strin[:steps]
    
def rotate(direction,anchor,strin):
    pos = re.search(anchor,strin).start()
    if pos >= 4:
        steps = pos + 2
    else:
        steps = (pos + 1)
    steps = steps % len(strin)
    return rotatedir(direction,steps,strin)
# abcde
#
# 0 -- 1 -- eabcd -- 1 -- 1
# 1 -- 2 -- deabc -- 3 -- 2
# 2 -- 3 -- cdeab -- 0 -- 
#
#
def rotate2(direction,anchor,strin):
    pos = re.search(anchor,strin).start()
    #print pos
    if pos == 1:
        steps = 1
    elif pos == 3:
        steps = 2
    elif pos == 5:
        steps = 3
    elif pos == 7:
        steps = 4
    elif pos == 2:
        steps = 6
    elif pos == 4:
        steps = 7
    elif pos == 6:
        steps = 8
    elif pos == 0:
        steps = 1
    
    steps = steps % len(strin) 
    return rotatedir(direction,steps,strin)

def reverse(positions,strin):
    posx = sorted(positions)[0]
    posy = sorted(positions)[1]
    substr = strin[posx:posy+1]
    if (posy-posx) >= len(strin)-1:
        return strin[::-1]
    else:
        ### Bug on the last idx
        newstr = strin[0:posx]  + substr[::-1]
        #### HOOOOOOW
        if posy < len(strin)-1:
            newstr += strin[-(len(strin)-posy-1):]
        return newstr
    
def move(positions,strin):
    posx = (positions)[0]
    posy = (positions)[1]
    substr = strin[0:posx] + strin[posx+1:]
    return substr[0:posy] + strin[posx] + substr[posy:]

In [69]:
data = pd.read_csv('./input.txt',header=None,delimiter='\n')
commands = [line[0].split(' ') for line in data.values]

strin = 'abcdefgh'
print 'unscrambled: ' + strin
for command in commands:
   # print command
    if command[0] == 'rotate' and command[1] == 'based':
        strin = rotate('right',command[6],strin)
    elif command[0] == 'rotate' and 'step' in command[3]:
        strin = rotatedir(command[1],int(command[2]),strin)
    elif command[0] == 'swap' and command[1] == 'position':
        strin = swapchar((int(command[2]),int(command[5])),strin)
    elif command[0] == 'swap' and command[1] == 'letter':
        strin = swapletter(command[2],command[5],strin)
    elif command[0] == 'reverse':
        strin = reverse((int(command[2]),int(command[4])),strin)
    elif command[0] == 'move':
        strin = move((int(command[2]),int(command[5])),strin)
    
print 'scrambled:   ' + strin

print '#############################################################'
strin = 'fbgdceah'
print 'scrambled:   ' + strin
for command in reversed(commands):
   # print command
   # print strin
    if command[0] == 'rotate' and command[1] == 'based':
        strin = rotate2('left',command[6],strin)
    elif command[0] == 'rotate' and 'step' in command[3]:
        if command[1] == 'left':
            direction = 'right'
        else:
            direction = 'left'
            
        strin = rotatedir(direction,int(command[2]),strin)
    elif command[0] == 'swap' and command[1] == 'position':
        strin = swapchar((int(command[5]),int(command[2])),strin)
    elif command[0] == 'swap' and command[1] == 'letter':
        strin = swapletter(command[5],command[2],strin)
    elif command[0] == 'reverse':
        strin = reverse((int(command[2]),int(command[4])),strin)
    elif command[0] == 'move':
        strin = move((int(command[5]),int(command[2])),strin)
    #print strin
    
print 'unscrambled: ' + strin

unscrambled: abcdefgh
scrambled:   ghfacdbe
#############################################################
scrambled:   fbgdceah
unscrambled: fhgcdaeb


In [60]:
strin = 'abcde'

strin = swapchar((4,0),strin)
print 'swap: ' + strin
strin =  swapletter('d','b',strin)
print 'swap: ' + strin
strin = reverse((0,4),strin)
print 'reverse: ' + strin
strin = rotatedir('left',1,strin)
print 'rotate: ' + strin
strin = move((1,4),strin)
print 'move ' + strin
strin = move((3,0),strin)
print 'move ' + strin
strin = rotate('b',strin)
print 'rotate ' + strin
strin = rotate('d',strin)
print 'rotate ' + strin

swap: ebcda
swap: edcba
reverse: abcde
rotate: bcdea
move bdeac
move abdec


TypeError: rotate() takes exactly 3 arguments (2 given)

In [171]:
data = pd.read_csv('./input.txt',header=None,delimiter='\n')
commands = [line[0].split(' ') for line in data.values]

blocked= []
for command in commands:
    bsplit = command[0].split('-')
    blocked.append((int(bsplit[0]),int(bsplit[1])))
    
blocked = sorted(blocked, key=lambda x: x[0])

lower = blocked[0][0]
upper = blocked[0][1]

num_free = 0
min_value = 0
max_int =0xFFFFFFFF

for bounds in blocked:
    # m------m
    #              m-------m
    ## non overlapping
    if bounds[0] > (upper+1):
        if min_value == 0:
            min_value = upper+1
        #add values between bounds[0] and upper
        num_free += bounds[0] - upper -1
        lower = bounds[0]
        upper = bounds[1]
    # m-------m
    #     m----------m
    elif bounds[0] <= upper and bounds[1] >= upper:
        upper = bounds[1]
    # m-------m
    #         m-------m
    elif bounds[0] == upper or bounds[0] == upper+1:
        upper = bounds[1]
        
if upper < max_int-1:
    num_free += max_int-upper-1
print "minimum: " + str(min_value)
print "# of free: " + str(num_free)

minimum: 23923783
# of free: 125
